In [ ]:
# NOTE: This is a revised version of your CDS hedging system with:
# - Rolling monthly training window (12 months)
# - Dynamic notional scaling and optimization
# - Visualizations for hedge actions, instruments, and allocations
# - All metrics in millions
# - Optional Bloomberg or CSV data source

import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from itertools import combinations
from xbbg import blp

# ----------------------------
# Configuration
# ----------------------------
TICKERS = {
    'CDX IG': 'CDX IG CDSI GEN 5Y Corp',
    'CDX HY': 'CDX HY CDSI GEN 5Y Corp',
    'iTraxx Main': 'ITRAXX EUR CDSI GEN 5Y Corp',
    'iTraxx Xover': 'ITRAXX XOVER CDSI GEN 5Y Corp'
}

COUPON_RATES = {'CDX IG': 100, 'CDX HY': 500, 'iTraxx Main': 100, 'iTraxx Xover': 500}
CS01_LIMITS = {'CDX IG': 0.25, 'CDX HY': 0.10, 'iTraxx Main': 0.25, 'iTraxx Xover': 0.10}  # in millions
NOTIONAL = 10  # in millions
WINDOW_MONTHS = 12

# ----------------------------
# Load Data
# ----------------------------
def fetch_cds_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    for name, bbg_ticker in tickers.items():
        df = blp.bdh(bbg_ticker, 'PX_LAST', start_date, end_date, Per='W')
        df = df.rename(columns={'PX_LAST': name})
        data = pd.concat([data, df], axis=1)
    return data.dropna()

def load_data(pnl_file, cds_file=None):
    pnl_data = pd.read_csv(pnl_file, parse_dates=['date']).set_index('date')
    pnl_data /= 1_000_000
    pnl_weekly = pnl_data.resample('W-FRI').sum()

    if cds_file:
        cds_data = pd.read_csv(cds_file, parse_dates=['date']).set_index('date')
    else:
        cds_data = fetch_cds_data(TICKERS, '2021-01-01', '2024-12-31')

    data = pd.concat([pnl_weekly, cds_data], axis=1).dropna()
    return data.iloc[:, 0], data.iloc[:, 1:]

# ----------------------------
# Optimization & Model
# ----------------------------
def calculate_cs01(spread, notional):
    return notional * 5 * 0.0001  # 5Y duration assumption

def optimize_hedge(expected_pnl, spreads, cs01_vals, cs01_lims, tickers):
    roll_cost = 0.2 * spreads.values * NOTIONAL / 52 * 0.0001
    coupon_cost = np.array([COUPON_RATES[t] * NOTIONAL / 52 * 0.0001 for t in tickers])
    total_cost = roll_cost + coupon_cost

    def objective(weights):
        return -np.dot(weights, expected_pnl - total_cost)

    constraints = [{'type': 'ineq', 'fun': lambda w, i=i: cs01_lims[i] - w[i] * cs01_vals[i]} for i in range(len(tickers))]
    bounds = [(0, 2) for _ in tickers]  # scaling up to 2x notional
    result = minimize(objective, np.zeros(len(tickers)), bounds=bounds, constraints=constraints)
    return result.x if result.success else np.zeros(len(tickers))

# ----------------------------
# Backtest & Run
# ----------------------------
def run_strategy(pnl_series, cds_data):
    returns = cds_data.pct_change().dropna()
    pnl_series = pnl_series[returns.index]
    monthly_idx = returns.resample('M').last().index

    records = []
    weights_df = pd.DataFrame(index=monthly_idx, columns=TICKERS.keys())
    signal_series = pd.Series(index=monthly_idx)
    combo_series = pd.Series(index=monthly_idx)

    for month in monthly_idx:
        end_date = month
        start_date = end_date - pd.DateOffset(months=WINDOW_MONTHS)
        train_mask = (returns.index >= start_date) & (returns.index < end_date)
        X, y = returns[train_mask], pnl_series[train_mask]
        if len(X) < 20: continue

        reg = RandomForestRegressor().fit(X, y)
        pred_pnl = pd.Series(reg.predict(X), index=X.index)

        best_combo, best_weights, best_pnl, best_hedge_pnl = None, None, None, None
        best_sharpe = -np.inf

        for r in [1, 2]:
            for combo in combinations(TICKERS.keys(), r):
                subX = X[list(combo)]
                hedge_pnl = -subX * NOTIONAL * 0.5
                avg_hedge = hedge_pnl.mean().values
                latest_spreads = cds_data.loc[end_date][list(combo)]
                cs01_vals = [calculate_cs01(latest_spreads[t], NOTIONAL) for t in combo]
                cs01_lims = [CS01_LIMITS[t] for t in combo]
                weights = optimize_hedge(avg_hedge, latest_spreads, cs01_vals, cs01_lims, combo)

                cost = 0.2 * latest_spreads.values * NOTIONAL / 52 * 0.0001 + \
                       np.array([COUPON_RATES[t] * NOTIONAL / 52 * 0.0001 for t in combo])
                net_pnl = pred_pnl + hedge_pnl @ weights - weights @ cost

                sharpe = net_pnl.mean() / (net_pnl.std() + 1e-6)
                if sharpe > best_sharpe:
                    best_combo, best_weights, best_pnl, best_hedge_pnl = combo, weights, net_pnl, hedge_pnl @ weights
                    best_sharpe = sharpe

        signal_model = RandomForestClassifier().fit(X[list(best_combo)], (best_pnl > 0).astype(int))
        pred_signal = signal_model.predict(X[list(best_combo)])[-1]

        weights_series = pd.Series({k: 0 for k in TICKERS})
        weights_series[list(best_combo)] = best_weights
        weights_df.loc[month] = weights_series
        signal_series.loc[month] = pred_signal
        combo_series.loc[month] = ' + '.join(best_combo)
        records.append(best_pnl[-1])

    results = pd.DataFrame({'Net PnL': records}, index=monthly_idx[:len(records)])
    return results, signal_series, weights_df.fillna(0), combo_series

# ----------------------------
# Dashboard
# ----------------------------
def display_dashboard(results, signals, weights_df, combo_series):
    st.set_page_config(layout="wide")
    st.title("📈 Dynamic CDS Hedging Strategy")

    st.subheader("🚦 Buy/Sell Signal Timeline")
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(results.index, results['Net PnL'], label='Net PnL', color='blue')
    ax.scatter(results.index[signals == 1], results['Net PnL'][signals == 1], color='green', label='BUY Hedge', marker='^')
    ax.scatter(results.index[signals == 0], results['Net PnL'][signals == 0], color='red', label='SELL Hedge', marker='v')
    ax.legend(); st.pyplot(fig)

    st.subheader("📌 Hedge Product Selected")
    st.line_chart(combo_series)

    st.subheader("📊 Hedge Notional Allocation Over Time")
    fig2, ax2 = plt.subplots(figsize=(10, 4))
    weights_df.astype(float).plot.area(ax=ax2)
    ax2.set_ylabel("Notional (millions)")
    st.pyplot(fig2)

    st.subheader("📈 Net PnL Over Time")
    st.line_chart(results['Net PnL'])

# ----------------------------
# Main
# ----------------------------
if __name__ == '__main__':
    st.sidebar.header("Upload Files")
    pnl_file = st.sidebar.file_uploader("Upload Desk PnL CSV", type=["csv"])
    cds_file = st.sidebar.file_uploader("Optional: Upload CDS Spread CSV", type=["csv"])

    if pnl_file:
        pnl_series, cds_data = load_data(pnl_file, cds_file)
        results, signals, weights_df, combo_series = run_strategy(pnl_series, cds_data)
        display_dashboard(results, signals, weights_df, combo_series)
    else:
        st.info("Upload PnL CSV file to begin analysis")
